#Ejercicio complementario Arranque en frío

Instalación de paqueterías no incluidas en Google colab


In [ ]:
pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 27.5 MB/s eta 0:00:00


In [ ]:
pip install opendatasets


In [ ]:
pip install fuzzywuzzy

Importar las librerías necesarias

In [ ]:
import opendatasets as od
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.sparse import csr_matrix
import implicit

Descargar el dataset


In [ ]:
od.download(
	"https://www.kaggle.com/datasets/andrewmvd/spotify-playlists")

Skipping, found downloaded files in "./spotify-playlists" (use force=True to force download)


Lectura del dataset y eliminación de observaciones no utiles

In [ ]:
#leer el dataset
df = pd.read_csv("spotify-playlists//spotify_dataset.csv",on_bad_lines='skip')

#Renombrar las columnas
df.columns = ['user', 'artist', 'song', 'playlist']

#Eliminar todos aquellos valores NA
df.dropna(inplace=True)

#Eliminar todos los valores duplicados
df.drop_duplicates(inplace=True)

#Eliminar las observaciones que presenten los siguientes valores en la columna de playlist
#Estas observaciones se eliminan porque no son playlist publicas
df.drop(df[df['playlist']=='Starred'].index,inplace=True)
df.drop(df[df['playlist']=='Liked from Radio'].index,inplace=True)
df.drop(df[df['playlist']=='Favoritas de la radio'].index,inplace=True)
df.drop(df[df['playlist']=='New Playlist'].index,inplace=True)
df.drop(df[df['playlist']=='My Shazam Tracks'].index,inplace=True)
df.drop(df[df['playlist']=='All Live Files'].index,inplace=True)
df.drop(df[df['playlist']=='iPhone'].index,inplace=True)


#obtener un Id para cad usuario
user_id = {}
cnt = 1
for user in df['user']:
    if user not in user_id:
        user_id[user] = cnt
        cnt+=1

cnt = 1
playlist_id = {}
#Obtener el conteo de las playlist
playlists = {}
for playlist in df['playlist']:
    if playlist in playlists:
      playlists[playlist] += 1
    else:
      playlists[playlist] = 1
      playlist_id[playlist] = cnt
      cnt += 1

Generación de columnas útiles

In [ ]:
#Anadir la dataset las columnas de ID y de reporducciones por playlist
df['User_id'] = df['user'].apply(lambda use: user_id[use])
df['Repros'] = df['playlist'].apply(lambda playlist: playlists[playlist])
df['playlist_id'] = df['playlist'].apply(lambda playlist: playlist_id[playlist])



In [ ]:
df.head()

,user,artist,song,playlist,User_id,Repros,playlist_id
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010,1,67,1
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010,1,67,1
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010,1,67,1
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010,1,67,1
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010,1,67,1


Generar dataset con el número de reproducciones que tiene playlist

In [ ]:
#Generar un dataset que solamente contenga el titulo de playlist y su numero de reproducciones
playlist_df = pd.DataFrame.from_dict(playlists,orient='index')
playlist_df.columns = ['Repros']
playlist_df.reset_index(inplace = True)
playlist_df.columns = ['Playlist','Repros']
playlist_df.sort_values(by='Repros',ascending=False,inplace=True)
playlist_df.reset_index(inplace = True,drop=True)

Obtener solamente las 20 playlist con mas reproducciones en el dataset


In [ ]:
toMatain = playlist_df.loc[:20]
del playlist_df

In [ ]:
toMatain

,Playlist,Repros
0,Rock,30107
1,2014,22674
2,Christmas,22236
3,2013,20870
4,Work,18408
5,Jazz,18266
6,Indie,17858
7,Classical,16328
8,everything,15705
9,Country,15503


Eliminar todas aquellas observaciones que no posean alguna playlist de las mostradas anteriormente.

In [ ]:
df = df[df['playlist'].isin(toMatain['Playlist'])]

Realizar el mismo proceso que se lleve a cabo con las playlist, pero ahora con los artistas.

In [ ]:
#Obtener las reproducciones por artista
ReprosArtistas = {}
for artist in df['artist']:
  if artist in ReprosArtistas:
    ReprosArtistas[artist]+=1
  else:
    ReprosArtistas[artist]=1

In [ ]:
artists_df = pd.DataFrame.from_dict(ReprosArtistas,orient='index')

In [ ]:
artists_df.reset_index(inplace=True)
artists_df.columns = ['artist','repros']
artists_df.sort_values(by='repros',ascending=False,inplace=True)
artists_df.reset_index(inplace=True)

toMatain = artists_df.loc[:20]
df = df[df['artist'].isin(toMatain['artist'])]

In [ ]:
df.reset_index(inplace=True,drop=True)
df

,user,artist,song,playlist,User_id,Repros,playlist_id
0,50346e4190d1707ebc6b39a95f86927a,Miles Davis,Basin Street Blues,Jazz,14,18266,208
1,50346e4190d1707ebc6b39a95f86927a,Miles Davis,I Fall in Love Too Easily,Jazz,14,18266,208
2,50346e4190d1707ebc6b39a95f86927a,Miles Davis,My Funny Valentine,Jazz,14,18266,208
3,23a17802002bcd5ce26954efcbed2676,Johann Sebastian Bach,"Cello Suite No. 1 in G Major, BWV 1007: Prelude",Classical,30,16328,466
4,23a17802002bcd5ce26954efcbed2676,Wolfgang Amadeus Mozart,"Die Zauberflöte (The Magic Flute), K. 620, Act...",Classical,30,16328,466
...,...,...,...,...,...,...,...
28583,bf7fd0e14a76a1945d1bd0cce485fd26,Miles Davis,Water on the Pond,Jazz,15066,18266,208
28584,bf7fd0e14a76a1945d1bd0cce485fd26,Miles Davis,Weirdo,Jazz,15066,18266,208
28585,bf7fd0e14a76a1945d1bd0cce485fd26,Miles Davis,When Lights Are Low,Jazz,15066,18266,208
28586,bf7fd0e14a76a1945d1bd0cce485fd26,Miles Davis,Woody 'N You - Live Version,Jazz,15066,18266,208


In [ ]:
dfForModel = df.loc[:,['User_id','playlist_id']].groupby(['User_id', 'playlist_id']).size().reset_index(name='rating')

In [ ]:
dfForModel.head()

,User_id,playlist_id,rating
0,14,208,3
1,30,466,13
2,91,1337,30
3,98,466,8
4,98,1388,3


Generar la sparse matrix

In [ ]:
def create_X(df: pd.DataFrame):
    N = df['User_id'].nunique()
    M = df['playlist_id'].nunique()

    user_mapper = dict(zip(np.unique(df["User_id"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["playlist_id"]), list(range(M))))

    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["User_id"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["playlist_id"])))

    user_index = [user_mapper[i] for i in df['User_id']]
    movie_index = [movie_mapper[i] for i in df['playlist_id']]

    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))

    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

In [ ]:
X, user_mapper, playlist_mapper, user_inv_mapper, playlist_inv_mapper = create_X(dfForModel)

In [ ]:
from fuzzywuzzy import process

def movie_finder(playlist):
    all_playlist = df['playlist'].tolist()
    df.head()
    closest_match = process.extractOne(playlist,all_playlist)
    return closest_match[0]

playlist_title_mapper = dict(zip(df['playlist'], df['playlist_id']))
playlist_title_inv_mapper = dict(zip(df['playlist_id'], df['playlist']))

def get_movie_index(playlist):
    fuzzy_title = movie_finder(playlist)
    playlist_id = playlist_title_mapper[fuzzy_title]
    playlist_idx = playlist_mapper[playlist_id]
    return playlist_idx

def get_movie_title(playlist_idx):
    playlist_id = playlist_inv_mapper[playlist_idx]
    playlist = playlist_title_inv_mapper[playlist_id]
    return playlist

Generar instancia del modelo

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=50)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


Entrenar modelo

In [ ]:
model.fit(X.T.tocsr())

  0%|          | 0/15 [00:00<?, ?it/s]

Obtener Recomendaciones de el genero Jazz


In [ ]:
playlist_of_interest = 'Jazz'

playlist_index = get_movie_index(playlist_of_interest)
related = model.similar_items(playlist_index)
related

(array([ 0, 15,  1, 12, 11,  5,  3,  4, 18, 20], dtype=int32),
 array([0.99999994, 0.08077116, 0.02867347, 0.0229955 , 0.02293672,
        0.02138848, 0.01213473, 0.00943337, 0.00422794, 0.00344394],
       dtype=float32))

In [ ]:
print(f"Por que miraste la pelicula de {movie_finder(playlist_of_interest)} te pueden interesar las siguientes peliculas:")
for t, r in zip(related[0], related[1]):

    recommended_title = get_movie_title(t)
    if recommended_title != movie_finder(playlist_of_interest):
        print(recommended_title)


Por que miraste la pelicula de Jazz te pueden interesar las siguientes peliculas:
Rock
Classical
Work
Chill
2012
2014
Pop
Music
Generos


In [ ]:
user_id = 91

In [ ]:
user_ratings = dfForModel[dfForModel['User_id']==user_id].merge(df[['playlist_id', 'playlist']])
user_ratings = user_ratings.sort_values('rating', ascending=False)
top_5 = user_ratings.head()
top_5

,User_id,playlist_id,rating,playlist
0,91,1337,30,Random
105,91,1337,30,Random
98,91,1337,30,Random
99,91,1337,30,Random
100,91,1337,30,Random


In [ ]:
X_t = X.T.tocsr()
user_idx = user_mapper[user_id]
recommendations = model.recommend(user_idx, X_t[user_idx])
recommendations

(array([ 7,  1,  0,  8, 14, 13,  4, 19, 11,  5], dtype=int32),
 array([5.07718883e-03, 1.16644241e-03, 5.19296154e-04, 3.44058499e-04,
        1.46264210e-04, 1.30685978e-04, 2.37030908e-05, 1.42725185e-05,
        9.47527587e-06, 2.26031989e-06], dtype=float32))

In [ ]:
for t, r in zip(recommendations[0], recommendations[1]):
    recommended_title = get_movie_title(t)
    print(recommended_title)

Christmas
Classical
Jazz
Rap
Country
Metal
Pop
everything
Chill
2012
